# Modulariser la conception du fichier d’index

## Ébauche de l’algorithme

Comme le script prend de l’ampleur, il est temps de réfléchir à l’organisation du code. Pour cela, une opération s’impose : réfléchir à l’algorithme du programme.

En énumérant les actions, on parviendra à dégager les besoins puis à repérer les opérations redondantes.

Pour chaque script :
- récupérer les métadonnées
- remplir l’index

Format de l’index ?
- une ligne par script
- titre, n° saison, n° épisode, transcripteur, propriétaire

![Organigramme ](./images/schema-index.png)

Quels sont les besoins ?
- identifier les trois premiers métas à partir du titre du fichier
- lire les scripts
- récupérer le méta *transcripteur* dans la dernière ligne du script (si présent)
- rajouter le méta *propriétaire* (Hypnoweb)
- écrire le résultat dans un fichier

À partir des besoins recensés, l’algorithme de création du fichier d’index pourrait ressembler à :
1. établir la liste des scripts
2. pour chaque script :
    - récupérer les premières métadonnées
    - lire la dernière ligne
    - identifier le transcripteur et le rajouter aux métadonnées
3. rajouter le propriétaire aux métadonnées
4. Créer un fichier index.txt :
    - pour chaque ligne de métadonnées :
        - pour chaque métadonnée :
            - écrire la métadonnée puis une virgule
        - passer à la ligne

## Découper en modules

L’enchaînement des actions permet de dégager les opérations qui ne sont pas mises à disposition de manière native par le langage :
- récupérer les métadonnées
- identifier le transcripteur
- rajouter une métadonnée à un ensemble de métadonnées

Pour ces opérations, le plus simple est d’écrire des fonctions dans un module (p.ex. `tools.py`).

### Récupérer les métadonnées à partir du nom de fichier

In [ ]:
# Module commun à toutes les fonctions utilisateur
import re

In [ ]:
def get_metadata(filename):
    """Defines metadata from a filename and pushes them into an array
    
    Keyword argument:
    filename -- the filename to parse
    """
    output = []
    pattern = 'S(?P<season>[0-9]{2})E(?P<episode>[0-9]{2,3})-(?P<title>.+).txt'
    metas = re.match(pattern, filename)
    output.append(metas.group('title'))
    output.append(metas.group('season'))
    output.append(metas.group('episode'))
    return output

### Identifier le transcripteur

In [ ]:
def get_alias(sentence):
    """Regex that looks in a sentence for the alias of a transcriber
    
    Keyword argument:
    sentence -- the string in which look for an alias
    """
    if sentence.startswith('Rédigé par'):
        pattern = 'Rédigé par\s?(?P<alias>[A-Za-z1-9]+)\s?.+'
        m = re.match(pattern, sentence)
        return m.group('alias')
    else:
        return ''

### Rajouter une métadonnée à un ensemble

In [ ]:
def add_meta(what, where, in_which):
    """Adds a metadata at a specific index in an array of metadata
    
    Keyword arguments:
    what -- the object to record
    where -- the index where to record the object
    in_which -- the array of metadata to update
    """
    in_which[where].append(what)

## Écrire la procédure principale

Importer la liste des modules :

In [ ]:
# Modules to import
import modules.tools as tools, os

Établir la liste des scripts :

In [ ]:
screenplays = os.listdir('./sample')

Initialiser une liste vide de métadonnées :

In [ ]:
metadata = []

Pour chaque script : obtenir les premières métadonnées, identifier le transcripteur et rajouter la mention du propriétaire :

In [ ]:
# For each screenplay
for index, screenplay in enumerate(screenplays):
    # Fetches metadata
    metadata.append(tools.get_metadata(screenplay))
    # Opens the screenplay
    with open(f'./sample/{screenplay}') as file:
        # Focus on the last line
        last_line = file.readlines()[-1]
        # Gets the alias
        alias = tools.get_alias(last_line)
        # Adds the alias in the metadata array
        tools.add_meta(alias, index, metadata)
    # Finally, adds the owner of the script
    tools.add_meta('Hypnoweb', index, metadata)

Créer un fichier `index.txt` :

In [ ]:
# Writes the index.txt file
with open('./files/index.txt', 'a') as file:
    # For each block of metadata…
    for block in metadata:
        # … same treatment for each metadata
        for i, meta in enumerate(block):
            # with comma between each, except for the last one
            file.write(f'{meta},') if i < (len(block) - 1) else file.write(meta)
        # Line break
        file.write('\n')